In [ ]:
import numpy as np
from matplotlib import pyplot as plt
from scipy import integrate

from control_theory import utils
from control_theory import systems

In [ ]:
pend = systems.PendODESystem()

In [ ]:
theta = utils.get_control(pend.A, pend.b)

In [ ]:
start, stop = 0, 10

time = np.linspace(start, stop, 300)
y_0 = np.array([0, 0.1, 0, 0])

sol = integrate.solve_ivp(
    systems.linear_system,
    (start, stop),
    y_0,
    dense_output=True,
    args=(pend.A, pend.b, theta),
    method="RK45"
)

In [ ]:
z = sol.sol(time)

In [ ]:
y_labels = (r"x", r"\phi", r"\dot x", r"\dot \phi")
# plt.rcParams['text.usetex'] = True # uncomment if you have latex

fig, axs = plt.subplots(4, 1)
fig.set_size_inches(10, 15)

for i in range(4):
    axs[i].plot(time, z[i])
    axs[i].set_xlabel('time')
    axs[i].set_ylabel(y_labels[i])
    axs[i].grid(True)

fig.tight_layout()
# fig.savefig('out.png', dpi=300, facecolor='white') # uncomment to save high-res picture
plt.show()